In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

t = symbols('t') #time(s)
pi = np.pi
m = 1.355 #mass of drone (kg)
l = 0.28 #arm length from centroid
g = 9.81 #acceleration due to gravity (m/s^2)

τ0,τ1,τ2,τ3 = symbols('τ0,τ1,τ2,τ3') #torque of each motor (N*m)
τ = Matrix([τ0,τ1,τ2,τ3]) #set of torque magnitudes
ω = 165*Matrix([1/τ0,1/τ1,1/τ2,1/τ3]) #rotor speeds from torques (rad/s) given by max power draw over torque
T = (9.81/(5630*pi))*Matrix([30*ω[0] - 3988*pi, 30*ω[1] - 3988*pi, 30*ω[2] - 3988*pi, 30*ω[3] - 3988*pi]) #thrusts from rotor speeds (N)

ΣT = T[0] + T[1] + T[2] + T[3] #net thrust (N)
kT = ΣT/(ω[0]**2 + ω[1]**2 + ω[2]**2 + ω[3]**2) #thrust coefficient (kg*m/rad^2)
I = Matrix([[0.018545,0,0],[0,0.018545,0],[0,0,0.0361185]]) #moment of inertia of entire drone (kg*m^2) ## REDO WITH VALUES FROM CAD
Ir = 0.00006430625 #moment of inertia of one rotor (kg*m^2), calculated approximating propeller as a disc and rotor as a cylinder

x = Function('x')(t) #translational degree of freedom, x (m), with respect to time
y = Function('y')(t) #translational degree of freedom, y (m), with respect to time
z = Function('z')(t) #translational degree of freedom, z (m), with respect to time
θ = Function('θ')(t) #rotational degree of freedom, θ (rad), with respect to time
ψ = Function('ψ')(t) #rotational degree of freedom, ψ (rad), with respect to time
φ = Function('φ')(t) #rotational degree of freedom, φ (rad), with respect to time

s = Matrix([φ,θ,ψ,x,y,z]) #state vector containing each degree of freedom
S = [s[0],s[1],s[2],s[3],s[4],s[5]] #list of all degrees of freedom
ds = s.diff(t) #time derivative of state vector, containing 3 angular and 3 translational velocities
dds = ds.diff(t) #second time derivative of state vector, containing 3 angular and 3 translational accelerations

ddφ = (1/I[0,0])*((I[1,1] - I[2,2])*ds[1]*ds[2] - Ir*ds[1]*(ω[0] - ω[1] + ω[2] - ω[3]) + l*kT*(ω[3]**2 - ω[1]**2)) #angular acceleration about roll axis
ddθ = (1/I[1,1])*((I[2,2] - I[0,0])*ds[2]*ds[0] - Ir*ds[0]*(ω[0] - ω[1] + ω[2] - ω[3]) + l*kT*(ω[0]**2 - ω[2]**2)) #angular acceleration about pitch axis
ddψ = (1/I[2,2])*((I[0,0] - I[1,1])*ds[0]*ds[1]) #angular acceleration about yaw axis
ddx = (ΣT/m)*(sin(θ)*cos(ψ)*cos(φ) + sin(ψ)*sin(φ)) #acceleration in x direction
ddy = (ΣT/m)*(sin(θ)*sin(ψ)*cos(φ) - cos(ψ)*sin(φ)) #acceleration in y direction
ddz = (ΣT/m)*(cos(θ)*cos(φ) - m*g/ΣT) #acceleration in z direction

ODEs = Matrix([ddφ,ddθ,ddψ,ddx,ddy,ddz]) #system of ODEs of second derivatives
Sys = dds - ODEs #Dynamical system characterized by our system of ODEs for the second derivatives of the state vector
DySys = [Sys[0],Sys[1],Sys[2],Sys[3],Sys[4],Sys[5]] #list of ODEs

State = dsolve(DySys,S)

acon = [symbols('C1'),symbols('C2'),symbols('C3'),symbols('C4'),symbols('C5'),symbols('C6'),symbols('C7'),symbols('C8'),symbols('C9'),symbols('C10'),symbols('C11'),symbols('C12')] #arbitrary constants
icon = [symbols('dθ0'),symbols('φ0'),symbols('dψ0'),symbols('θ0'),symbols('dφ0'),symbols('ψ0'),symbols('dx0'),symbols('x0'),symbols('dy0'),symbols('y0'),symbols('dz0'),symbols('z0')] #symbolic initial conditions

xt = State[3].rhs.subs(acon,icon) #Functional form of x(t)
yt = State[4].rhs.subs(acon,icon) #Functional form of y(t)
zt = State[5].rhs.subs(acon,icon) #Functional form of z(t)
φt = State[0].rhs.subs(acon,icon) #Functional form of φ(t)
θt = State[1].rhs.subs(acon,icon) #Functional form of θ(t)
ψt = State[2].rhs.subs(acon,icon) #Functional form of ψ(t)

def toFunc(dof, τorques, icons):
    ics = τorques
    ics.extend(icons) #extends the list of controlled torques by the list of initial conditions for a full set of initial values
    sic = [τ0,τ1,τ2,τ3]
    sic.extend(icon) #extended list of symbolic initial conditions
    return dof.subs(sic,ics)

def toVals(fun, time):
    return fun.subs(t, time) #returns the numerical value of one degree of freedom at a specific time

ic1 = [0,0,0,0,0,0,0,0,0,0,0,1]
ic1 = [0,-0.102,0.25,0,0,0,0.5,0,0,-2,0,1]
ic31 = [0,0,0,0,0,0,0,0,0,0,0.1,0]
ic32 = [0,0,0,0,0,0,1,0,0,0,0,1]
ic33 = [0,0,pi/20,0,0,0,0,5,0,0,0,1]
ic34 = [0,0,0,0,0,pi/2,0,5,1,0,0,1]
ic35 = [0,0,0,0,0,0,0,5,0,5,-0.01,1]

iτ1 = [0.266822,0.266822,0.266822,0.266822] #hover torques
iτ2 = [0.266590,0.266590,0.266590,0.266590] #circle yaw torques
iτ31 = [0.266822,0.266822,0.266822,0.266822] #hover torques
iτ32 = [0.266822,0.266822,0.266822,0.266822] #hover torques
iτ33 = [0.266822,0.266822,0.266822,0.266822] #flat yaw torques
iτ34 = [0.266822,0.266822,0.266822,0.266822] #hover torques
iτ35 = [0.266822,0.266822,0.266822,0.266822] #hover torques

xf1 = toFunc(xt,iτ1,ic1) #creates a function of x(t) for the first performance goal, given initial conditions
yf1 = toFunc(yt,iτ1,ic1) #creates a function of y(t) for the first performance goal, given initial conditions
zf1 = toFunc(zt,iτ1,ic1) #creates a function of z(t) for the first performance goal, given initial conditions
φf1 = toFunc(φt,iτ1,ic1) #creates a function of φ(t) for the first performance goal, given initial conditions
θf1 = toFunc(θt,iτ1,ic1) #creates a function of θ(t) for the first performance goal, given initial conditions
ψf1 = toFunc(ψt,iτ1,ic1) #creates a function of ψ(t) for the first performance goal, given initial conditions

xf2 = toFunc(xt,iτ2,ic2) #creates a function of x(t) for the second performance goal, given initial conditions
yf2 = toFunc(yt,iτ2,ic2) #creates a function of y(t) for the second performance goal, given initial conditions
zf2 = toFunc(zt,iτ2,ic2) #creates a function of z(t) for the second performance goal, given initial conditions
φf2 = toFunc(φt,iτ2,ic2) #creates a function of φ(t) for the second performance goal, given initial conditions
θf2 = toFunc(θt,iτ2,ic2) #creates a function of θ(t) for the second performance goal, given initial conditions
ψf2 = toFunc(ψt,iτ2,ic2) #creates a function of ψ(t) for the second performance goal, given initial conditions

xf31 = toFunc(xt,iτ31,ic31) #creates a function of x(t) for the first part of the third performance goal, given initial conditions - Launching
yf31 = toFunc(yt,iτ31,ic31) #creates a function of y(t) for the first part of the third performance goal, given initial conditions - Launching
zf31 = toFunc(zt,iτ31,ic31) #creates a function of z(t) for the first part of the third performance goal, given initial conditions - Launching
φf31 = toFunc(φt,iτ31,ic31) #creates a function of φ(t) for the first part of the third performance goal, given initial conditions - Launching
θf31 = toFunc(θt,iτ31,ic31) #creates a function of θ(t) for the first part of the third performance goal, given initial conditions - Launching
ψf31 = toFunc(ψt,iτ31,ic31) #creates a function of ψ(t) for the first part of the third performance goal, given initial conditions - Launching

xf32 = toFunc(xt,iτ32,ic32) #creates a function of x(t) for the second part of the third performance goal, given initial conditions - Straight line
yf32 = toFunc(yt,iτ32,ic32) #creates a function of y(t) for the second part of the third performance goal, given initial conditions - Straight line
zf32 = toFunc(zt,iτ32,ic32) #creates a function of z(t) for the second part of the third performance goal, given initial conditions - Straight line
φf32 = toFunc(φt,iτ32,ic32) #creates a function of φ(t) for the second part of the third performance goal, given initial conditions - Straight line
θf32 = toFunc(θt,iτ32,ic32) #creates a function of θ(t) for the second part of the third performance goal, given initial conditions - Straight line
ψf32 = toFunc(ψt,iτ32,ic32) #creates a function of ψ(t) for the second part of the third performance goal, given initial conditions - Straight line

xf33 = toFunc(xt,iτ33,ic33) #creates a function of x(t) for the third part of the third performance goal, given initial conditions - Yawing
yf33 = toFunc(yt,iτ33,ic33) #creates a function of y(t) for the third part of the third performance goal, given initial conditions - Yawing
zf33 = toFunc(zt,iτ33,ic33) #creates a function of z(t) for the third part of the third performance goal, given initial conditions - Yawing
φf33 = toFunc(φt,iτ33,ic33) #creates a function of φ(t) for the third part of the third performance goal, given initial conditions - Yawing
θf33 = toFunc(θt,iτ33,ic33) #creates a function of θ(t) for the third part of the third performance goal, given initial conditions - Yawing
ψf33 = toFunc(ψt,iτ33,ic33) #creates a function of ψ(t) for the third part of the third performance goal, given initial conditions - Yawing

xf34 = toFunc(xt,iτ34,ic34) #creates a function of x(t) for the fourth part of the third performance goal, given initial conditions - Straight line
yf34 = toFunc(yt,iτ34,ic34) #creates a function of y(t) for the fourth part of the third performance goal, given initial conditions - Straight line
zf34 = toFunc(zt,iτ34,ic34) #creates a function of z(t) for the fourth part of the third performance goal, given initial conditions - Straight line
φf34 = toFunc(φt,iτ34,ic34) #creates a function of φ(t) for the fourth part of the third performance goal, given initial conditions - Straight line
θf34 = toFunc(θt,iτ34,ic34) #creates a function of θ(t) for the fourth part of the third performance goal, given initial conditions - Straight line
ψf34 = toFunc(ψt,iτ34,ic34) #creates a function of ψ(t) for the fourth part of the third performance goal, given initial conditions - Straight line

xf35 = toFunc(xt,iτ35,ic35) #creates a function of x(t) for the fifth part of the third performance goal, given initial conditions - Landing
yf35 = toFunc(yt,iτ35,ic35) #creates a function of y(t) for the fifth part of the third performance goal, given initial conditions - Landing
zf35 = toFunc(zt,iτ35,ic35) #creates a function of z(t) for the fifth part of the third performance goal, given initial conditions - Landing
φf35 = toFunc(φt,iτ35,ic35) #creates a function of φ(t) for the fifth part of the third performance goal, given initial conditions - Landing
θf35 = toFunc(θt,iτ35,ic35) #creates a function of θ(t) for the fifth part of the third performance goal, given initial conditions - Landing
ψf35 = toFunc(ψt,iτ35,ic35) #creates a function of ψ(t) for the fifth part of the third performance goal, given initial conditions - Landing

t1 = np.linspace(0, 120, 1200) #2 minute hover
t2 = np.linspace(0, 60, 600) #1 minute circle
t31 = np.linspace(0, 10, 100) #10 second launch
t32 = np.linspace(10, 15, 50) #5 second forward
t33 = np.linspace(15, 25, 100) #10 second yaw
t34 = np.linspace(25, 30, 50) #5 second forward
t35 = np.linspace(30, 130, 1000) #10 second land
t3 = np.linspace(0,130,1300) #full performance goal 3

def fill(f,tp):
    fp = []
    for i in range(len(tp)):
        fp.append(toVals(f,tp[i]))
    return fp

xp1 = fill(xf1, t1)
yp1 = fill(yf1, t1)
zp1 = fill(zf1, t1)
φp1 = fill(φf1, t1)
θp1 = fill(θf1, t1)
ψp1 = fill(ψf1, t1)

xp2 = fill(xf2, t2)
yp2 = fill(yf2, t2)
zp2 = fill(zf2, t2)
φp2 = fill(φf2, t2)
θp2 = fill(θf2, t2)
ψp2 = fill(ψf2, t2)

xp31 = fill(xf31, t31)
yp31 = fill(yf31, t31)
zp31 = fill(zf31, t31)
φp31 = fill(φf31, t31)
θp31 = fill(θf31, t31)
ψp31 = fill(ψf31, t31)

xp32 = fill(xf32, t32)
yp32 = fill(yf32, t32)
zp32 = fill(zf32, t32)
φp32 = fill(φf32, t32)
θp32 = fill(θf32, t32)
ψp32 = fill(ψf32, t32)

xp33 = fill(xf33, t33)
yp33 = fill(yf33, t33)
zp33 = fill(zf33, t33)
φp33 = fill(φf33, t33)
θp33 = fill(θf33, t33)
ψp33 = fill(ψf33, t33)

xp34 = fill(xf34, t34)
yp34 = fill(yf34, t34)
zp34 = fill(zf34, t34)
φp34 = fill(φf34, t34)
θp34 = fill(θf34, t34)
ψp34 = fill(ψf34, t34)

xp35 = fill(xf35, t35)
yp35 = fill(yf35, t35)
zp35 = fill(zf35, t35)
φp35 = fill(φf35, t35)
θp35 = fill(θf35, t35)
ψp35 = fill(ψf35, t35)

xp31.extend(xp32)
xp31.extend(xp33)
xp31.extend(xp34)
xp31.extend(xp35)

yp31.extend(yp32)
yp31.extend(yp33)
yp31.extend(yp34)
yp31.extend(yp35)

zp31.extend(zp32)
zp31.extend(zp33)
zp31.extend(zp34)
zp31.extend(zp35)

φp31.extend(φp32)
φp31.extend(φp33)
φp31.extend(φp34)
φp31.extend(φp35)

θp31.extend(θp32)
θp31.extend(θp33)
θp31.extend(θp34)
θp31.extend(θp35)

ψp31.extend(ψp32)
ψp31.extend(ψp33)
ψp31.extend(ψp34)
ψp31.extend(ψp35)

xp3 = xp31
yp3 = yp31
zp3 = zp31
φp3 = φp31
θp3 = θp31
ψp3 = ψp31

P1 = [φp1,θp1,ψp1,xp1,yp1,zp1]
P2 = [φp2,θp2,ψp2,xp2,yp2,zp2]
P3 = [φp3,θp3,ψp3,xp3,yp3,zp3]

variables = [φ,θ,ψ,x,y,z]

fig, axs = plt.subplots(6, 3, figsize=(15, 10))

for i in range(6):
    axs[i].plot(t1, P1[i])
    axs[i].set_ylim([-pi,pi])
    axs[i].set_xlabel('Time (s)')
    axs[i].set_ylabel(variables[i])
    axs[i].grid(True)

for i in range(6):
    axs[i+6].plot(t2, P2[i])
    axs[i+6].set_ylim([-pi,pi])
    axs[i+6].set_xlabel('Time (s)')
    axs[i+6].set_ylabel(variables[i])
    axs[i+6].grid(True)

for i in range(6):
    axs[i+12].plot(t3, P3[i])
    axs[i+12].set_ylim([-pi,pi])
    axs[i+12].set_xlabel('Time (s)')
    axs[i+12].set_ylabel(variables[i])
    axs[i+12].grid(True)

plt.tight_layout()
plt.show()

Matrix([
[ -15.0984092747371*(27225/τ3**2 - 27225/τ1**2)*(-27.7955808170515 + 2.74546517196124/τ3 + 2.74546517196124/τ2 + 2.74546517196124/τ1 + 2.74546517196124/τ0)/(27225/τ3**2 + 27225/τ2**2 + 27225/τ1**2 + 27225/τ0**2) + 0.00346757886222702*(-165/τ3 + 165/τ2 - 165/τ1 + 165/τ0)*Derivative(θ(t), t) + 0.947613912105689*Derivative(θ(t), t)*Derivative(ψ(t), t) + Derivative(φ(t), (t, 2))],
[-15.0984092747371*(-27225/τ2**2 + 27225/τ0**2)*(-27.7955808170515 + 2.74546517196124/τ3 + 2.74546517196124/τ2 + 2.74546517196124/τ1 + 2.74546517196124/τ0)/(27225/τ3**2 + 27225/τ2**2 + 27225/τ1**2 + 27225/τ0**2) + 0.00346757886222702*(-165/τ3 + 165/τ2 - 165/τ1 + 165/τ0)*Derivative(φ(t), t) + Derivative(θ(t), (t, 2)) - 0.947613912105689*Derivative(φ(t), t)*Derivative(ψ(t), t)],
[                                                                                                                                                                                                                                      